# SVM pipeline for flux analysis and permutation flux analysis

## Parameter definition

In [1]:
results_path = "path"
features = "genes" # "FVA", "genes"
permute = "_label_permutation" # "", "_label_permutation", "_feature_permutation";  "" if you do not want permutation

NUMBER_REPETITIONS = 200
NUMBER_TEST_SAMPLES = 5 # per class
NUMBER_TRAIN_SAMPLES = 30 # per class per fold execution
NUMBER_INTERNAL_FOLDS = 5
var_threshold = 0.0 # 0.0 to remove only constant features
classifier = "SVM" # "SVM", "NN", "RF"

## Data loading and minimal preprocessing

In [2]:
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.io import loadmat, savemat

random_state = 1
random.seed(random_state)
rng = np.random.RandomState(random_state)

# load dataset
if features == "FVA":
    data = pd.read_csv("path\maxFluxes.csv")
    # load metabolic model
    model = loadmat("path\recon2.2.mat")
    rxn_list = model["model"][0][0][0]
    pathway_list = model["model"][0][0][13]
    # check for NANs and for identical rows
    print(data.isnull().values.any())
    print(data.duplicated(subset=data.columns[:-1]).any())
    sample_ids = data["sample_id"]
    data = data[data.columns[1:]]
    
    data = data.round(decimals=6)   # done here because it is element-wise so no data leakage can happen
    print(data.isnull().values.any())                        
    print(data.duplicated(subset=data.columns[:-1]).any())
else:
    data = loadmat("path\combined_dataset.mat")
    data = np.transpose(data["geneData_combat_p"])
    print(np.sum(np.isnan(data)))
    uni_data, _, unique_inverse = np.unique(data, return_index=True, return_inverse=True, axis=0)
    print((uni_data[unique_inverse] != data).any()) # because rows are sorted after np.unique
    gene_list = pd.read_csv("path\gene_exp_final_m.csv")
    gene_list = gene_list["hgnc_id"].to_numpy()
    
    data = np.around(data, decimals=6)
    print(np.sum(np.isnan(data)))
    uni_data, _, unique_inverse = np.unique(data, return_index=True, return_inverse=True, axis=0)
    print((uni_data[unique_inverse] != data).any())   

    
targets = pd.read_csv("path\sampleData.csv")  # ids are in the same order
targets = targets["Status"] == "Tumor"
targets = LabelEncoder().fit_transform(targets)

0
False
0
False


## Util definition

In [3]:
# custom transformer for removing the identical features
from sklearn.base import BaseEstimator, TransformerMixin

class RedundantTrimmer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        _, unique_indices = np.unique(X, return_index=True, axis=1)  #  sorted, unlike in the MATLAB version
        self.indices = unique_indices
        return self
    
    def transform(self, X, y=None):
        return X[:,self.indices]
    

# function to retrieve the rxns and subsystems used
def getFeatures(features, thresholded, redundant):
    features = features.reshape(-1, 1)
    features = features[remove_indices]
    unique_red = list(set(redundant))
    array = np.zeros(len(unique_red), dtype=object)
    for i in unique_red:
        array[i] = [features[j] for j, k in enumerate(redundant) if k == i]
    return array

## Model building and validation analyses

In [4]:
from sklearn.utils import shuffle
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, matthews_corrcoef

if features == "FVA":
    tumor_samples = data.to_numpy()[targets==1,:]
    healthy_samples = data.to_numpy()[targets==0,:]
else:
    tumor_samples = data[targets==1,:]
    healthy_samples = data[targets==0,:]
cv = StratifiedKFold(n_splits=NUMBER_INTERNAL_FOLDS, shuffle=True, random_state=random_state)

if classifier == "SVM": 
    mdl = SVC(kernel="linear", random_state=rng)
    params = {
        "model__C": np.logspace(-10, 10, num=10)
    }
elif classifier == "NN":
    mdl = MLPClassifier(solver="lbfgs", random_state=rng)
    params = {
        "model__alpha": 10.0 ** -np.arange(1, 7)
    }
else:
    mdl = RandomForestClassifier(random_state=rng)
    params = {
        "model__n_estimators": [10, 100, 500, 1000],
        "model__max_depth": [3, 5, 8, None]
    }

pipe = Pipeline(steps=[
    ("oversampling", RandomOverSampler(sampling_strategy="minority", random_state=random_state)), 
    ("undersampling", RandomUnderSampler(sampling_strategy={0:NUMBER_TRAIN_SAMPLES, 1:NUMBER_TRAIN_SAMPLES}, 
                                         random_state=random_state)),
    ("var_selection", VarianceThreshold(threshold=var_threshold)), 
    ("reduntant_trim", RedundantTrimmer()), 
    ("standardise", StandardScaler()), 
    ('model',  mdl)
])

params = {
    "svc__C": lambdas
}

accuracies = np.zeros(NUMBER_REPETITIONS)
MCCs = np.zeros(NUMBER_REPETITIONS)
Ytrue = np.zeros((NUMBER_REPETITIONS, NUMBER_TEST_SAMPLES * 2))
Ypred = np.zeros((NUMBER_REPETITIONS, NUMBER_TEST_SAMPLES * 2))
testSamples = np.zeros((NUMBER_REPETITIONS, NUMBER_TEST_SAMPLES * 2))
betas = list()
if features == "FVA":
    predictors =  np.zeros(NUMBER_REPETITIONS, dtype=object)
    pathways =  np.zeros(NUMBER_REPETITIONS, dtype=object)
else:
    genes =  np.zeros(NUMBER_REPETITIONS, dtype=object)

for rep in range(NUMBER_REPETITIONS):
    test_idx1 = random.sample(range(len(healthy_samples)), NUMBER_TEST_SAMPLES)
    test_idx2 = random.sample(range(len(tumor_samples)), NUMBER_TEST_SAMPLES)  
    testSamples[rep] = np.concatenate((test_idx1, test_idx2)) + 1 # for use in MATLAB
        
    Xtest = np.concatenate((healthy_samples[test_idx1,:], tumor_samples[test_idx2,:]))
    ytest = [i for i in range(2) for _ in range(NUMBER_TEST_SAMPLES)]
    
    train_healthy = np.delete(healthy_samples, test_idx1, axis=0)
    train_tumor = np.delete(tumor_samples, test_idx2, axis=0)
    Xtrain = np.concatenate((train_healthy, train_tumor))
    Xtrain, ytrain = shuffle(Xtrain, np.concatenate(([0] * len(train_healthy), [1] * len(train_tumor))), 
                             random_state=random_state)
    
    if permute == "_label_permutation":
        ys = np.concatenate((ytrain, ytest))
        random.shuffle(ys)
        ytrain, ytest = ys[:len(ytrain)], ys[-len(ytest):]
    elif permute == "_feature_permutation":
        raise NotImplementedError
    else:
        pass
    
    gs = GridSearchCV(pipe, param_grid=params, cv=cv, refit=True, n_jobs=5) 
    gs.fit(Xtrain, ytrain)  
    print("Run", rep + 1, gs.best_params_, gs.best_score_) 
    ypred = gs.predict(Xtest)
    accuracies[rep] = accuracy_score(ytest, ypred)
    MCCs[rep] = matthews_corrcoef(ytest, ypred)
    Ytrue[rep] = ytest
    Ypred[rep] = ypred
    
    Xtrain = Xtrain[gs.best_estimator_["oversampling"].sample_indices_, :]
    Xtrain = Xtrain[gs.best_estimator_["undersampling"].sample_indices_, :]
    remove_indices = gs.best_estimator_["var_selection"].get_support()
    Xtrain = Xtrain[:,remove_indices]
    _, _, collapse_indices = np.unique(Xtrain, return_index=True, return_inverse=1, axis=1)
    
    if features == "FVA":
        predictors[rep] = getFeatures(rxn_list, remove_indices, collapse_indices)
        pathways[rep] = getFeatures(pathway_list, remove_indices, collapse_indices)
    else:
        genes[rep] = getFeatures(gene_list, remove_indices, collapse_indices)

    if classifier == "SVM":                      
        betas.append({"Beta": gs.best_estimator_["model"].coef_[0]})
    elif classifier == "NN": 
        betas.append({"Beta": np.squeeze(gs.best_estimator_["model"].coefs_[0].dot(gs.best_estimator_["model"].coefs_[1]))})
    else:
        betas.append({"Beta": np.squeeze(np.zeros((Xtrain.shape[0], 1)))}) # too computational expensive to calculate feature
                                                                           # importance     

print("Average accuracy: {} with standard deviation: {}".format(np.mean(accuracies), np.std(accuracies)))
print("Average MCC: {} with standard deviation: {}".format(np.mean(MCCs), np.std(MCCs)))

Run 1 {'svc__C': 0.0004641588833612782} 0.624384236453202
Run 2 {'svc__C': 0.0004641588833612782} 0.6948275862068967
Run 3 {'svc__C': 0.0004641588833612782} 0.744088669950739


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Run 4 {'svc__C': 0.0004641588833612782} 0.566256157635468
Run 5 {'svc__C': 0.0004641588833612782} 0.6667487684729065
Run 6 {'svc__C': 1e-10} 0.6729064039408867
Run 7 {'svc__C': 0.0004641588833612782} 0.6950738916256156
Run 8 {'svc__C': 0.0004641588833612782} 0.6948275862068966
Run 9 {'svc__C': 1e-10} 0.5958128078817735
Run 10 {'svc__C': 0.0004641588833612782} 0.7019704433497537
Run 11 {'svc__C': 0.0004641588833612782} 0.8009852216748768


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Run 12 {'svc__C': 0.0004641588833612782} 0.6741379310344828
Run 13 {'svc__C': 0.0004641588833612782} 0.7086206896551724
Run 14 {'svc__C': 0.0004641588833612782} 0.6810344827586208
Run 15 {'svc__C': 0.0004641588833612782} 0.6812807881773401
Run 16 {'svc__C': 0.0004641588833612782} 0.7305418719211823
Run 17 {'svc__C': 0.0004641588833612782} 0.6812807881773398
Run 18 {'svc__C': 0.0004641588833612782} 0.6588669950738917


...

Run 192 {'svc__C': 1e-10} 0.7027093596059114
Run 193 {'svc__C': 0.0004641588833612782} 0.6029556650246305
Run 194 {'svc__C': 0.0004641588833612782} 0.6600985221674878
Run 195 {'svc__C': 0.0004641588833612782} 0.6655172413793103


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Run 196 {'svc__C': 0.0004641588833612782} 0.6667487684729064
Run 197 {'svc__C': 0.0004641588833612782} 0.6820197044334975
Run 198 {'svc__C': 0.0004641588833612782} 0.6238916256157635
Run 199 {'svc__C': 0.0004641588833612782} 0.7088669950738916
Run 200 {'svc__C': 1e-10} 0.5330049261083744


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Average accuracy: 0.633 with standard deviation: 0.1712045560141435
Average MCC: 0.0017851544578240076 with standard deviation: 0.31436534176836983


## Result saving

In [5]:
if classifier == "SVM": 
    SVM_results = {
        "predicted": Ypred, 
        "true": Ytrue, 
        "mdl": betas, 
        "accuracies": accuracies, 
        "MCCs": MCCs, 
        "testSamples": testSamples
    }

    if features == "FVA":
        SVM_results["predictors"] = predictors
        SVM_results["pathways"] = pathways
    else:
        SVM_results["genes"] = genes
    
    savemat(results_path + r"\SVMl2" + "_results_" + features + permute + ".mat", 
            {"SVM_results": SVM_results}, oned_as="column")
elif classifier == "NN": 
    NN_results = {
        "predicted": Ypred, 
        "true": Ytrue, 
        "mdl": betas, 
        "accuracies": accuracies, 
        "MCCs": MCCs, 
        "testSamples": testSamples
    }

    if features == "FVA":
        NN_results["predictors"] = predictors
        NN_results["pathways"] = pathways
    else:
        NN_results["genes"] = genes
    savemat(results_path + r"\NN" + "_results_" + features + permute + ".mat", 
            {"NN_results": NN_results}, oned_as="column")
else: 
    RF_results = {
        "predicted": Ypred, 
        "true": Ytrue, 
        "mdl": betas, 
        "accuracies": accuracies, 
        "MCCs": MCCs, 
        "testSamples": testSamples
    }

    if features == "FVA":
        RF_results["predictors"] = predictors
        RF_results["pathways"] = pathways
    else:
        RF_results["genes"] = genes
    
    savemat(results_path + r"\RF" + "_results_" + features + permute + ".mat", 
            {"RF_results": RF_results}, oned_as="column")